### total 12,447개

미국 - 1042개(뉴스)  
미국 - 1094개(블로그)  

여행 - 991개(뉴스)  
여행 - 1098개(블로그)  

네이버 - 815개(뉴스)  
네이버 - 1944개(블로그)  

네이버블로그 - 453개(뉴스)  
네이버블로그 - 975개(블로그)  

네이버쇼핑 - 300개(뉴스)  
네이버쇼핑 - 1000개(블로그)  

네이버지도 - 508개(뉴스)  
네이버지도 - 924개(블로그)  

네이버클라우드 - 376개(뉴스)  
네이버클라우드 - 927개(블로그)  

***import***

In [1]:
import numpy as np
import pandas as pd
import pykospacing
spacer = pykospacing.Spacing()
from konlpy.tag import Okt
okt = Okt()
import re
from gensim import corpora
import gensim
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()

**데이터 불러오기**

In [2]:
NAVERCLOUD_news = pd.read_csv('네이버클라우드(뉴스)-376row.csv')
NAVERCLOUD_blog = pd.read_csv('네이버클라우드(블로그)-927row.csv')

In [3]:
NAVERCLOUD_news

,title,link,description
0,"현대오토에버, 클라우드 강화…네이버 출신 정수환 상무 영입",https://n.news.naver.com/mnews/article/015/000...,\n\n클라우드기술사업부장 신규 선임\n\n\n\n정수환 신임 현대오토에버 클라우드...
1,"현대오토에버, 네이버클라우드 출신 정수환 상무 영입",https://n.news.naver.com/mnews/article/001/001...,\n\n\n\n\n\n정수환 현대오토에버 클라우드기술사업부장(상무)[현대오토에버 제...
2,"현대오토에버, 네이버클라우드 출신 정수환 상무 영입",https://n.news.naver.com/mnews/article/018/000...,\n\n\n\n\n\n정수환 현대오토에버 클라우드기술사업부장 상무(사진=현대오토에버...
3,"현대오토에버, 네이버클라우드 출신 정수환 상무 영입",https://n.news.naver.com/mnews/article/366/000...,\n\n\n\n\n\n정수환 상무./현대오토에버 제공 현대오토에버는 클...
4,"현대오토에버, 클라우드 사업 총괄에 '네이버 출신' 정수환 상무",https://n.news.naver.com/mnews/article/008/000...,\n\n\n\n\n\n정수환 현대오토에버 클라우드기술사업부장(상무)./사진제공=현대...
...,...,...,...
371,"네이버클라우드, 글로벌 개인정보보호 인증 'APEC CBPR' 취득",https://n.news.naver.com/mnews/article/092/000...,"\n\n네이버 이어 국내 2번째이자 CSP 최초…""글로벌 비즈니스 확장 박차""네이버..."
372,"네이버클라우드, 국내 CSP 최초 글로벌 개인정보보호 인증 'APEC CBPR' 취득",https://n.news.naver.com/mnews/article/076/000...,\n\n\t\t\t네이버클라우드가 국내 클라우드 서비스 사업자(CSP) 최초로 아시...
373,"네이버클라우드, 글로벌 개인정보보호 인증 취득… 국내 CSP 최초",https://n.news.naver.com/mnews/article/020/000...,\n\n\n\n\n\n네이버클라우드는 국내 클라우드 서비스 사업자(CSP) 최초로 ...
374,"네이버클라우드, 글로벌 개인정보보호 인증 ‘APEC CBPR’ 취득",https://n.news.naver.com/mnews/article/138/000...,\n\n\n\n\n\n[디지털데일리 이종현기자] 네이버클라우드는 국내 클라우드 서비...


### 전처리 함수 
*6주차 01.WordCloud 실습코드 참고*

- <span style = 'font-size:1.1em;line-height:1.5em'>(1) 줄바꿈 문자 제거</span>
- <span style = 'font-size:1.1em;line-height:1.5em'>(2) 오탈자 교정 (생략)</span>
- <span style = 'font-size:1.1em;line-height:1.5em'>(3) 띄어쓰기 교정</span>
- <span style = 'font-size:1.1em;line-height:1.5em'>(4) sentence tokenization (너무 오래걸려서 생략)</span>
- <span style = 'font-size:1.1em;line-height:1.5em'>(5) 특수문자 문자 제거</span>
- <span style = 'font-size:1.1em;line-height:1.5em'>(6) word tokenization (형태소 분석 및 stemming)</span>
- <span style = 'font-size:1.1em;line-height:1.5em'>(7) stopword removal</span>

**불용어 목록 생성**

In [4]:
with open('stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = f.readlines()
stopwords = [x.replace('\n','') for x in stopwords] # stopword 파일의 줄바꿈 문자 제거

In [5]:
def preprocessing(x):
    p = re.compile('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]')
    results = []
    result = []
    x = x.replace('\n',' ') # (1) 줄바꿈 문자 제거
    df_news = spacer(x) # (3) 띄어쓰기 교정
    
    tokens = okt.morphs(x, stem=True) # (6) word tokenization
    for token in tokens:
        token = p.sub('', token) # (5) 특수문자 제거
        if token not in stopwords:
            result.append(token) # (7) stopword removal
    results.extend(result)
    results = ' '.join(results)
    return results

### pos_taging 함수

#(1) 품사 태깅 수행 

#(2) 명사, 형용사, 부사에 해당하는 단어들만 추출하여 저장

In [6]:
def pos_taging(x):
    pos_news = okt.pos(x) # (1)
    filtered_result = [word for word, pos in pos_news if pos in 'Noun'] # (2)
    nouns_news = [n for n in filtered_result if len(n) > 1]
    return nouns_news

### 전처리 및 postag

In [7]:
NAVERCLOUD_news['preprocessed_description'] = NAVERCLOUD_news['description'].apply(lambda x: preprocessing(x))
NAVERCLOUD_news['preprocessed_description'] = NAVERCLOUD_news['preprocessed_description'].apply(lambda x: pos_taging(x))

In [22]:
NAVERCLOUD_blog['preprocessed_description'] = NAVERCLOUD_blog['description'].apply(lambda x: preprocessing(x))
NAVERCLOUD_blog['preprocessed_description'] = NAVERCLOUD_blog['preprocessed_description'].apply(lambda x: pos_taging(x))

In [23]:
NAVERCLOUD_news['preprocessed_description']

0      [클라우드, 기술, 사업, 부장, 신규, 선임, 정수환, 신임, 현대, 오토, 에버...
1      [정수환, 현대, 오토, 에버, 클라우드, 기술, 사업, 부장, 상무, 현대, 오토...
2      [정수환, 현대, 오토, 에버, 클라우드, 기술, 사업, 부장, 상무, 사진, 현대...
3      [정수환, 상무, 현대, 오토, 에버, 제공, 오토, 에버, 클라우드, 기술, 사업...
4      [정수환, 현대, 오토, 에버, 클라우드, 기술, 사업, 부장, 상무, 사진, 제공...
                             ...                        
371    [네이버, 국내, 이자, 최초, 글로벌, 비즈니스, 확장, 박차, 네이버, 클라우드...
372    [네이버, 클라우드, 국내, 클라우드, 서비스, 사업자, 최초, 아시아, 태평양, ...
373    [네이버, 클라우드, 국내, 클라우드, 서비스, 사업자, 최초, 아시아, 태평양, ...
374    [디지털, 일리, 이종현, 기자, 네이버, 클라우드, 국내, 클라우드, 서비스, 사...
375    [국내, 네이버, 두번째, 최초, 네이버, 클라우드, 국내, 클라우드, 서비스, 사...
Name: preprocessed_description, Length: 376, dtype: object

### BOW(Bag-of-Words) 생성

In [24]:
word_dict_news = corpora.Dictionary(NAVERCLOUD_news['preprocessed_description'])

In [25]:
word_dict_blog = corpora.Dictionary(NAVERCLOUD_blog['preprocessed_description'])

In [26]:
corpus_news = [word_dict_news.doc2bow(text) for text in NAVERCLOUD_news['preprocessed_description']]

In [27]:
corpus_blog = [word_dict_blog.doc2bow(text) for text in NAVERCLOUD_blog['preprocessed_description']]

In [28]:
corpus_news[2]

[(0, 1),
 (1, 1),
 (2, 2),
 (4, 1),
 (5, 1),
 (6, 2),
 (7, 6),
 (8, 2),
 (9, 1),
 (10, 1),
 (11, 3),
 (12, 2),
 (14, 1),
 (15, 1),
 (16, 3),
 (17, 1),
 (18, 1),
 (19, 1),
 (20, 5),
 (21, 5),
 (22, 2),
 (23, 8),
 (26, 1),
 (27, 1),
 (28, 2),
 (32, 2),
 (34, 1),
 (35, 6),
 (36, 2),
 (37, 1),
 (38, 1),
 (39, 1),
 (40, 6),
 (41, 1),
 (42, 1),
 (44, 2),
 (45, 1),
 (46, 3),
 (47, 1),
 (48, 1),
 (50, 1),
 (51, 1),
 (52, 1),
 (53, 2),
 (55, 5),
 (56, 2),
 (58, 1),
 (59, 2),
 (60, 1),
 (61, 1),
 (62, 1),
 (65, 1),
 (66, 1),
 (67, 1),
 (68, 14),
 (70, 1),
 (73, 1),
 (74, 5),
 (75, 1),
 (76, 1),
 (83, 1),
 (88, 1),
 (90, 1),
 (91, 1),
 (92, 1),
 (93, 1),
 (94, 1),
 (95, 1),
 (96, 1),
 (97, 1)]

In [29]:
[(idx,word) for idx, word in word_dict_news.items()]

[(0, '강화'),
 (1, '계획'),
 (2, '고객'),
 (3, '관계자'),
 (4, '구독'),
 (5, '구축'),
 (6, '글로벌'),
 (7, '기술'),
 (8, '네이버'),
 (9, '노력'),
 (10, '데이터'),
 (11, '모빌리티'),
 (12, '미래'),
 (13, '발표'),
 (14, '복구'),
 (15, '본부장'),
 (16, '부장'),
 (17, '비롯'),
 (18, '비즈니스'),
 (19, '빅데이터'),
 (20, '사업'),
 (21, '상무'),
 (22, '생태계'),
 (23, '서비스'),
 (24, '선임'),
 (25, '설명'),
 (26, '성공'),
 (27, '센터'),
 (28, '수행'),
 (29, '신규'),
 (30, '신임'),
 (31, '아마존'),
 (32, '안정'),
 (33, '어드밴스드'),
 (34, '업계'),
 (35, '에버'),
 (36, '역량'),
 (37, '역임'),
 (38, '역할'),
 (39, '영역'),
 (40, '오토'),
 (41, '요구'),
 (42, '운영'),
 (43, '웹서비스'),
 (44, '위해'),
 (45, '인공'),
 (46, '인프라'),
 (47, '일조'),
 (48, '일환'),
 (49, '자격'),
 (50, '재난'),
 (51, '재해'),
 (52, '전략'),
 (53, '전문가'),
 (54, '정보기술'),
 (55, '정수환'),
 (56, '제공'),
 (57, '주력'),
 (58, '지능'),
 (59, '지원'),
 (60, '집중'),
 (61, '차량'),
 (62, '총괄'),
 (63, '최고'),
 (64, '최근'),
 (65, '최선'),
 (66, '출신'),
 (67, '커넥티드카'),
 (68, '클라우드'),
 (69, '탑인'),
 (70, '특화'),
 (71, '티어'),
 (72, '파트너'),
 (73, '퍼블릭'),
 (74, '현대'),
 (75

In [30]:
[(idx,word) for idx, word in word_dict_blog.items()]

[(0, '가량'),
 (1, '가상'),
 (2, '가장'),
 (3, '가지'),
 (4, '간의'),
 (5, '개인'),
 (6, '검색'),
 (7, '결론'),
 (8, '경우'),
 (9, '계정'),
 (10, '고려'),
 (11, '고정'),
 (12, '공간'),
 (13, '공유'),
 (14, '과정'),
 (15, '관해'),
 (16, '구독'),
 (17, '구성'),
 (18, '구조'),
 (19, '국내'),
 (20, '그것'),
 (21, '그대로'),
 (22, '기기'),
 (23, '기능'),
 (24, '기본'),
 (25, '기존'),
 (26, '기회'),
 (27, '낫다'),
 (28, '네이버'),
 (29, '네트워크'),
 (30, '노트북'),
 (31, '녹화'),
 (32, '누리'),
 (33, '다운로드'),
 (34, '단순'),
 (35, '대부분'),
 (36, '데스크톱'),
 (37, '데이터'),
 (38, '도구로'),
 (39, '동기화'),
 (40, '동영상'),
 (41, '디바이스'),
 (42, '디스크'),
 (43, '때문'),
 (44, '라면'),
 (45, '라운드'),
 (46, '로그인'),
 (47, '리기'),
 (48, '마이'),
 (49, '매력'),
 (50, '매우'),
 (51, '매칭'),
 (52, '먼저'),
 (53, '멀티'),
 (54, '멤버십'),
 (55, '면서'),
 (56, '모든'),
 (57, '모바일'),
 (58, '목적'),
 (59, '무료'),
 (60, '무방'),
 (61, '무분별'),
 (62, '박스'),
 (63, '반대'),
 (64, '반면'),
 (65, '방법'),
 (66, '방식'),
 (67, '방지'),
 (68, '배터리'),
 (69, '백업'),
 (70, '버전'),
 (71, '버튼'),
 (72, '번호'),
 (73, '보고'),
 (74, '본문'),
 (75, '본인'),

In [31]:
len(word_dict_news)

4599

### LDA 모델 훈련

#(1) 3개의 topic

#(2) LDA 모델을 생성. 'id2word'에는 단어 사전을 전달하고, 학습 반복 횟수는 15로 설정

#(3) 생성된 모델에서 각 주제에 대한 상위 4개 단어를 출력


In [47]:
N_TOPICS = 3 # (1) 
ldamodel_news = gensim.models.ldamodel.LdaModel(corpus_news, num_topics = N_TOPICS, id2word=word_dict_news, passes = 15) # (2)

topics = ldamodel_news.print_topics(num_words=4) # (3)
for topic in topics:
    print(topic)

(0, '0.030*"네이버" + 0.029*"클라우드" + 0.019*"서비스" + 0.011*"활용"')
(1, '0.042*"네이버" + 0.033*"클라우드" + 0.015*"기술" + 0.014*"서비스"')
(2, '0.042*"클라우드" + 0.025*"네이버" + 0.015*"디지털" + 0.013*"데이터"')


In [48]:
N_TOPICS = 3 # (1)
ldamodel_blog = gensim.models.ldamodel.LdaModel(corpus_blog, num_topics = N_TOPICS, id2word=word_dict_blog, passes = 15) # (2)

topics = ldamodel_blog.print_topics(num_words=4) # (3)
for topic in topics:
    print(topic)

(0, '0.045*"클라우드" + 0.034*"네이버" + 0.024*"서비스" + 0.020*"플랫폼"')
(1, '0.051*"클라우드" + 0.043*"네이버" + 0.010*"사진" + 0.009*"교육"')
(2, '0.038*"클라우드" + 0.035*"네이버" + 0.013*"서비스" + 0.012*"기술"')


### LDA 시각화

In [49]:
vis_news = pyLDAvis.gensim_models.prepare(ldamodel_news, corpus_news, word_dict_news)
pyLDAvis.display(vis_news)

In [50]:
vis_blog = pyLDAvis.gensim_models.prepare(ldamodel_blog, corpus_blog, word_dict_blog)
pyLDAvis.display(vis_blog)